In [6]:
%%writefile plug_ai.slurm
#!/bin/bash
#SBATCH --job-name=plug_ai
#SBATCH --output=plug_ai.out
#SBATCH --error=plug_ai.out
#SBATCH --gres=gpu:1
#SBATCH --ntasks-per-node=1
#SBATCH --nodes=1
#SBATCH --hint=nomultithread
#SBATCH --time=00:50:00
#SBATCH --qos=qos_gpu-dev
#SBATCH --cpus-per-task=10
#SBATCH --account=sos@v100
#SBATCH -C v100-16g


## load Pytorch module
module purge

module load pytorch-gpu/py3/1.10.1

## launch script on every node
set -x

# code execution
srun python /gpfswork/rech/ibu/ssos023/Plug-AI/JZ_experiments/main.py \
--dataset_dir /gpfsscratch/idris/sos/ssos022/Medical/Task01_BrainTumour/ \
--limit_sample 20 --batch_size 2 --lr 0.0001

Overwriting plug_ai.slurm


In [7]:
%%bash
# submit job
sbatch plug_ai.slurm

Submitted batch job 389657


In [8]:
from threading import Event
import signal
from IPython.display import clear_output

def quit(signo, _frame):
    print("Interrupted by %d, shutting down" % signo)
    exit.set()

for sig in ('TERM', 'HUP', 'INT'):
    signal.signal(getattr(signal, 'SIG'+sig), quit);

exit = Event()

sq = !squeue -u $USER -n plug_ai
tail = !tail plug_ai.out
while len(sq) >= 2 and not exit.is_set():
    
    clear_output(wait=True)
    print(sq[0], sq[1], sep='\n')
    print(*tail, sep='\n')
    
    sq = !squeue -u $USER -n plug_ai
    tail = !tail plug_ai.out

    exit.wait(1)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            389657   gpu_p13  plug_ai  ssos023 CG       1:13      1 r10i1n7
Loading pytorch-gpu/py3/1.10.1
  Loading requirement: cuda/11.2 nccl/2.9.6-1-cuda cudnn/8.1.1.33-cuda gcc/8.4.1
    openmpi/4.1.1-cuda intel-mkl/2020.4 magma/2.5.4-cuda sox/14.4.2
+ srun python /gpfswork/rech/ibu/ssos023/Plug-AI/JZ_experiments/main.py --dataset_dir /gpfsscratch/idris/sos/ssos022/Medical/Task01_BrainTumour/ --limit_sample 20 --batch_size 2 --lr 0.0001
Final Args: {'mode': 'Training', 'dataset_dir': '/gpfsscratch/idris/sos/ssos022/Medical/Task01_BrainTumour/', 'task': 'THIS IS A TEST', 'n_class': 4, 'categories': ['cat0', 'cat1', 'cat2', 'cat3'], 'limit_sample': 20, 'batch_size': 2, 'nb_epoch': 1, 'learning_rate': '5e-05', 'device': 'cuda', 'random_seed': 2022, 'verbose': 'Full', 'export_config': True, 'config_file': '/gpfsdswork/projects/idris/sos/ssos023/Projects/Plug-AI/Plug-AI/JZ_experiments/config.yaml', 'lr

In [9]:
!scancel -u $USER -n plug_ai